# PyCUDA installation


In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4538988 sha256=2286999bc5c2806880ab36ca10e3f95ba46fd92f858e1e2f1602d169c42bdea6
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.2-py2.py3-none-any.whl size=62338 sha256=282f6e096e397cf6d83de7bd7e648936d564843df2f5e41f8b61761e24074769
  Stored in directory: /root/.cache/pip/wheels/a7/d6/ac/03a67d071bde6d272d1f7c9ab7f4344fa9d7b9d98bda7fd127
Successfully built pycuda pytools




---



# Version #1: using ```SourceModule```

PyCUDA initialization

In [ ]:
import numpy as np

# --- PyCUDA initialization
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


iDivUp function: if ```b``` divides ```a```, then ```a/b``` is returned, otherwise the function returns the integer division between ```a``` and ```b``` ```+1```.



In [ ]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a / b + 1) if (a % b != 0) else (a / b)

In [ ]:
########
# MAIN #
########

Defining two CUDA events that will be used to measure execution time.

In [ ]:
start = cuda.Event()
end   = cuda.Event()

Number of array elements

In [ ]:
N = 100000

Number of threads per block

In [ ]:
BLOCKSIZE = 256

Create two host vectors ```h_a``` and ```h_b``` of ```N``` random entries. ```np.random.randn``` returns ```float64```'s.

In [ ]:
h_a = np.random.randn(1, N)
h_b = np.random.randn(1, N)

Cast ```h_a``` and ```h_b``` to single precision (```float32```).

In [ ]:
h_a = h_a.astype(np.float32)
h_b = h_b.astype(np.float32)

Allocate ```h_a.nbytes```, ```h_b.nbytes``` and ```h_c.nbytes``` of GPU device memory space pointed to by ```d_a```, ```d_b``` and ```d_c```, respectively.

In [ ]:
d_a = cuda.mem_alloc(h_a.nbytes)
d_b = cuda.mem_alloc(h_b.nbytes)
d_c = cuda.mem_alloc(h_a.nbytes)

Copy the ```h_a``` and ```h_b``` arrays from host to the device arrays ```d_a``` and ```d_b```, respectively.

In [ ]:
cuda.memcpy_htod(d_a, h_a)
cuda.memcpy_htod(d_b, h_b)

Define the CUDA kernel function ```deviceAdd``` as a string. ```deviceAdd``` performs the elementwise summation of ```d_a``` and ```d_b``` and puts the result in ```d_c```.



In [ ]:
mod = SourceModule("""
  #include <stdio.h>
  __global__ void deviceAdd(float * __restrict__ d_c, const float * __restrict__ d_a, const float * __restrict__ d_b, const int N)
  {
    const int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;
    d_c[tid] = d_a[tid] + d_b[tid];
  }
  """)

Define a reference to the ```__global__``` function ```deviceAdd```.

In [ ]:
deviceAdd = mod.get_function("deviceAdd")

Define the block size ```blockDim``` and the grid size ```gridDim```.

In [ ]:
blockDim  = (BLOCKSIZE, 1, 1)
gridDim   = (int(iDivUp(N, BLOCKSIZE)), 1, 1)

Invoke the ```deviceAdd``` function.
Note that, up to here, ```N``` is an *object* of ```class int``` and not an integer number. Therefore, before using it, we must cast it to ```np.int32``` which is essentially the standard, single precision, floating point type.
Before launching ```deviceAdd```, the ```start``` and ```end``` events are recorded, so that the execution time can be measured.
Note that, before the processing time can be measured, all the activities in the current context must be ceased. This is the reason why ```end.synchronize()``` is invoked. Remember that the host and device executions are asynchronous. Furthermore, with the event record, the device will record a time stamp for the event when it reaches that event in the stream. Without synchronization, it happens that the ```end``` event is recorded after the ```deviceAdd``` function execution is actually terminated, as we expect, but the ```print``` function is executed before ```deviceAdd``` has actually finished its execution.


In [ ]:
# --- Warmup execution
deviceAdd(d_c, d_a, d_b, np.int32(N), block = blockDim, grid = gridDim)

start.record()
deviceAdd(d_c, d_a, d_b, np.int32(N), block = blockDim, grid = gridDim)
end.record()
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))

Processing time = 0.000107s


Allocate host space and copy results from device to host.

In [ ]:
h_c = np.empty_like(h_a)
cuda.memcpy_dtoh(h_c, d_c)

<class 'numpy.ndarray'>


Check if the device processing results are as expected.

In [ ]:
if np.array_equal(h_c, h_a + h_b):
  print("Test passed!")
else :
  print("Error!")

Test passed!


Finally, flush context printf buffer. Without flushing, no printout may be returned.

In [ ]:
cuda.Context.synchronize()